<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Nucleo F303RE Zero Hardware Board

# Board 01 : First Steps

This document describes the first steps to work with the SLab system when using the **STM32 NucleoF3403RE** hardware board. That includes programming the hardware board and setting up the Python environment.

In order to use this **notebook** you need to have a **STM32 Nucleo F3403RE** demonstration board.

Version 1.1 (16/3/2019)  

License information is at the end of the document

---
**Bill Of Materials (BOM):**

* STM32 NucleoF3403RE demonstration board
* USB Mini Communication cable

---

## Introduction

The **hardware board** is the basic instrument in the SLab Python project. It shall include, at least, two **DACs** and four **ADCs**. A **Digital to Analog Converter** (DAC) is a device that, given an input digital number, can generate a voltage output proportional to this number. An **Analog to Digital Converter** (ADC) is a device that, given a voltage in its input, measures it and returns a digital number that is proportional to the measured voltage. So, if we have a circuit to test, we can force two voltages using the two DACs and we can measure up to four node voltages using the four ADCs.

The board communicates with a PC using a **serial COM link**. Classic serial RS-232 links are not implemented in most modern PCs, so the serial connection is usually implemented inside an **USB** link. The board is usually powered by the PC so no external power supplies are required.

Any programmable microcontroller board that features a COM link, and, at least, two DACs and four ADCs, can be used as the hardware board if it is programmed with the proper firmware. An Arduino UNO board won't do, for instance, because although it features several ADCs, it doesn't include any DAC.

The board firmware basically waits for a command request from the PC. When a command is received it interacts with the ADCs and DACs depending on the request. On some requests, information is brought back to the PC when the command ends.

In this document we will use the [STM32 Nucleo64 F303RE](http://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-nucleo/nucleo-f303re.html) demonstration board, shown in the figure below, as the hardware board. As the full name is too long, we will call it the **F303RE Board** in the rest of this document. 

![NucleoF303RE Board](images\Board_01\F303RE.jpg)

This is an **official** SLab hardware board in the sense that a firmware is provided for it. **SLab** does not have any relationship with **ST**. We just use their board and provide a firmware. Consult the [SLab Board Folder](http://localhost:8888/notebooks/Boards) for up to date information of any other officially supported boards.  
As the communicatuion protocol is documented, you are free to use any non official board but, in this case, writing the firmware is up to you.

The [F303RE Board](http://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-nucleo/nucleo-f303re.html) is a microcontroller board that [ST](http://www.st.com/) provides as a way to demonstrate one of their ARM Cortex 32 bit microcontrollers, the [STM32F303RE](http://www.st.com/content/st_com/en/products/microcontrollers/stm32-32-bit-arm-cortex-mcus/stm32f3-series/stm32f303/stm32f303re.html). This microcontroller is quite interesting because is part of the [STM32F3](https://www.st.com/en/microcontrollers-microprocessors/stm32f3-series.html?querycriteria=productId=SS1576) fammily. Microcontrollers on this familly feature a 32-bit ARM Cortex M4 core (with FPU and DSP instructions) running at 72 MHz. The core is surrounded with a lot of peripherals some of them specifically designed for analog processing. The interesting peripherals respect to the **SLab** systems are:

* Up to four independent 12bit ADCs with 5 MSPS (Million Samples Per Second) capability
* Up to three 12bit DACs (some with analog drivers)
* Up to four embedded Operational Amplifiers

The ST manufacturer produces the Nucleo Board and sells them practically at cost price to promote their microcontrollers. ST produces, also, the Discovery series of boards that usually include a microcontroller and some peripherals. There are other families of boards produced by ST, but the Nucleo and Discovery series are the cheapest ones. 

There are three series of Nucleo boards, the Nucleo 32 , Nucleo 64 and Nucleo 144 that feature 32 pin, 64 in or 144 pin microcontrollers. The **F303RE board** is one the the intermediate 64 pin ones. All the Nucleo boards are [MBED](https://www.mbed.com/en/) enabled. That means that they all are programmable using a web based MBED compiler at its [developer site](https://developer.mbed.org/). So you don't need to install any toolchain to program a Nucleo board. In fact, the first versions of the SLab firmware were compiled using MBED. The current version of the **F303RE Board** firmware has been developed on [ChibiOS/RT](http://www.chibios.org/dokuwiki/doku.php), however.

If at any point you are tired of the SLab work, you can start learning about MCU programming using the F303RE board in the easy to set MBED environment.

An MBED enabled board also features an easy way to program the board with a binary firmware file. The board, upon connection through USB, is seen as a mass storage disk. Just **drag and drop** the firmware file over the shown disk and you are done.

## Connecting the board to the PC

In order to use the board, in our case, the F303RE Board, we need to connect it to a PC. In most cases it will be automatically detected and drivers will be loaded for if needed you can manually install the drivers before connecting the board.

The Nucleo boards use a ST-LINK/V2-1 interface to communicate with the PC. The Windows drivers can be located inside the [F303RE Board page](http://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-nucleo/nucleo-f303re.html). If you want, you can also obtain some installation information on the [MBED page](https://developer.mbed.org/platforms/ST-Nucleo-F303RE/) of the F303RE Board.

The USB protocol enables a connected system to be seen as several different devices at the same time. If all goes well, after connecting the board to the PC, it will be seen with three different interfaces:

* A mass storage device (like an external hard disk)
* A serial communication COM port
* A debug port

We will use the storage device just once to transfer the firmware to the board. Most of the time we will use the COM port as this is the link for the requests sent from the Python code to the hardware board. We won't never use the debug communication link. Neither is needed for normal web based MBED programming.

Most of the SLab work has been developed in a **Windows machine**, so no intensive checks have been performed on **Linux** and no test at all has been developed on **MAC** computers.

It has been tested that the drivers install automatically, at least for the mass storage and COM port on **Ubuntu Linux**. 

The Nucleo board includes two microcontrollers, one, a **STM32F103** next to the USB connector, implements the ST-LINK/V2-1 that communicates with the PC. And the second one, the bigger **STM32F303RE** 64 pin chip at the center of the board, is where we store our firmware. So, in fact, we have two firmwares in the board, one for each microcontroller.

ST releases from time to time new firmwares for the STM32F103 microcontroller that takes care of the ST-LINK connection. You can find an installer program for this firmware on the [F303RE board page](http://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-nucleo/nucleo-f303re.html). It is recommended to run this installer to guarantee that the STLINK firmware is up to date, but it never hurts to try without doing that. Anyway, upgrading the firmware is the first step to take if you experience any communication problem with the board. As far as I know this installer only works on Windows, so you would need a Windows system to upgrade the STLINK firmware.

# Uploading the SLab firmware

After the drivers installation, you shall see that the board is seen from the PC at least as a mass storage device, with name **"NODE_F303RE"** in the case of the F303RE Board, and as a COM serial link. In a Windows system you can see the COM port used in the device manager:

![win com](images\Board_01\win_com.png)

And also the mass storage device:

![win com](images\Board_01\win_storage.png)

You can also see both in the STM32 STLink properties under "Devices and Printers" :

![stlink properties](images\Board_01\stlink.png)

In a Linux machine, the board also is shown as a "NODE_F303RE" external storage unit and, in the machines tested, the serial is seen as **"/dev/ttyACM0"**.

Now it is time to upload the firmware on the board. For official boards, the firmware file has a name pattern:  
<BR>
    <center><B>SLAB-Board Name-X.Y(date).bin</B></center>
<BR>    
Where "Board Name" is the name of the board and X.Y are the major a minor versions of the firmware. As an example the filename of the firware for the F303 board released in SLab version 1.3 was:
<BR>
    <center><B>SLAB-Nucleo64-F303RE-1.2 (2018-02-11).bin</B></center>
<BR>

The firmware file can be located in the **SLab/Develop/SLab/Firmware/Bin** folder on the standard **SLab** distribution.

To program the firmware just drag and drop the file over the NODE_F303RE mass storage disk as if you where copying a file. You will see the board LED next to the USB connector blink during some time and is kept in still in green afterwards.

If all goes ok, you have successfully written the firmware. This procedure has been tested both in Windows and Linux machines.

## Testing the communication

If you want to verify that the firmware is loaded and that the communication link is in operation, you can use a terminal emulator like **Putty** for that. This is not mandatory so you can skip to next section if you want.

If you connect to the board using its COM port at **38400 baud, 8 bits, no parity**, you will see that every time you **reset** the board by hitting its **black button** the board identifies itself with its name like in the following figure:

![terminal](images\Board_01\terminal.png)

The text could be different in your case if you are using a newer firmware.

## Serial access in Linux

In a Linux machine you can use the screen program to access the serial port. Note, that you usually need to have root privileges to access the serial port. The following command has been tested to work to open the serial channel on a Linux Ubuntu machine:

>`> sudo screen /dev/ttyACM0 38400`

After entering the communication program, **reset** the board as in the Windows example and see if it responds with its firmware identification string.

Having to be **root** to use the serial port is a hassle. Moreover, it is a security issue because we will need to run **Python** as **root** and the use of **sudo** usually can conflict with your normal environment variables.

If you are in a Linux machine, it is recommended to be able to access to the serial ports without using Linux. One way to do that is removing the **modem manager** package and adding your user to the **dialup** group. The following two commands, executed from **bash** take care of that:

>`> sudo apt-get autoremove modemmanager`

>`> sudo adduser username dialout`

You will probably need to **reboot** for the changes to take effect, but you should now be able to use the serial ports without **sudo**.


After you are sure that all the hardware operates as expected and you can confidently go to the next software section.

## Setting up Python

As explained before, the SLab system use a **slab.py** module to communicate with the board. So you need Python to use the board. From **SLab** version **1.4** onwards, **Python 3.x** is the main test environment. 

The board relies on several modules, basically [PySerial](https://pythonhosted.org/pyserial/) to communicate with the board [NumPy](http://www.numpy.org/) to make calculations, [matplotlib](https://matplotlib.org/) to show graphs and [Jupyter](http://jupyter.org/) for the interactive documents. 

You can install Python from [python.org](https://www.python.org/) and add those packages or you can go the easy way and install a bundle that incorporates most needed packages like [Anaconda](https://www.continuum.io/downloads) that is available for Windows, Linux and Mac.

![Anaconda](images\Board_01\Anaconda.jpg)

For the current **SLab** instalation, Anaconda 3.x is the recommended one.

In **Windows** there are two executable file installers, one for 32 bits and other for 64 bits, just download the executable and run it to install **Anaconda** on the computer.

In **Linux** there are several versions depending on your **CPU** but they are all related to shell **.sh** files. Just download the proper **.sh** file and run it on your computer:

>`> bash filename.sh`

AFAIK, Anaconda can be installed in your user region without needing **root** access.

The installer asks if you want to uptdate the user **PATH**. In general it is recommended to say **yes**. That eases future access to Anaconda. The **vscode** integration that you can optionally install is not required at all for **SLab**.

In a proper Python installation you shall have in the path so that you can call the python interpreter from any place. If the installer does not do that for you, you will need to manually update the path information.

## Checking Pyserial

Anaconda always installs **numpy**, **matplotlib** and **Jupyter**, but is quite possible that **pyserial** is not automatically installed. You can check if **pyserial** is installed by opening a python interpreter by executing **python** form a shell prompt and trying to import this module. 

>`>>> import serial`

If Python goes silent without any error message, that means that **Pyserial** is installed on your system.

If you are running this document from **Jupyter**, you can also try to execute following **code** cell by selecting it and hitting the **Run** button.

In [ ]:
# Code to check if Pyserial is installed
print()
try:
    import serial
except:
    print("Pyserial is not installed, Use conda to install it: ")
    print()
    print("    conda install -c anaconda pyserial")
else:
    print("Pyserial is installed. Don't need to install it")
print()    

If **pyserial** is not installed, and you have installed **Anaconda**, you can use **conda** to install this module:

>`> conda install -c anaconda pyserial` 

Try to import the module after that to see if it works.

The **pyserial** is really the only **required** package to use **SLab** as it is needed to communicate with the hardware board. You can use SLab without having the **numpy** or **matplotlib** packages installed in Python. In that case the module will have reduced features. No plotting, for instance. You will be informed about that when you import SLab because it will show a message like:

>`Cannot load SciPy modules  
Functionality will be reduced`

It is not recommended to run **SLab** that way, however. It loses most of its magic.

## Linux on the hard way

If you have installed **Anaconda** or you plan to install it to ease your life, you can skip this section, if not, continue reading.

**Anaconda** is a really big package.

Perhaps you don't want all the packages or you are **geek** enough to do all installations yourself the hard way.

If you are running Linux, you probably have python already installed in your system. The rest of this section deals with Python 3.x so, you can test that it is installed by executing from the shell:

>`> python3`

Now you can check if the required packages are installed:

>`>>> import serial`  
>`>>> import numpy`  
>`>>> import matplotlib`  
>`>>> import jupyter`  

Alternativelly, if you running this document inside **Jupyter** you can check all the needed packages running the following script. Of course, **Jupyter** shall be installed for you to be able run the cell.

In [ ]:
import importlib

def check(package):
    try:
        importlib.import_module(package)
    except:
        print('Package',package,'is not installed (ERROR)')
    else:
        print('Package',package,'is installed (OK)')
        
check('serial')
check('numpy')
check('matplotlib')
check('jupyter')

If there is no error message on manual imports or all the mesages from the code cell say '(OK)', congratulations!, you don't need to install anything. If there is any import error, you will need to install the affected modules. If you want them that's it. You can, for instance, use **SLab** without **jupyter**, or, in fact **numpy** or **matplotlib**. The only bare minimum required package is **pyserial** for the communication.

>`python -m pip install pyserial`  
>`python -m pip install numpy`  
>`python -m pip install matplotlib`  
>`python -m pip install jupyter`

Of course, you need **pip** for that. If it is not installed, you would need to install it, for instance using:

>`sudo apt-get install python3-pip`

Having all the packages is not a guarantee for having a full working **SLab** environment. The environment will be tested later, but, as it directly relates to manual instalations, the explanations will be made here.

In the case of **matplotlib**, it relies on a [backend](https://matplotlib.org/1.3.0/faq/usage_faq.html#what-is-a-backend) to write the final plots. Depending on the current version of Python and Matplotlib you can have some issues. For instance, if the default backend is **Gtk3Agg** it could conflict with **Python3**.  
You know if this is the case if you try to run the following code:

In [ ]:
import numpy as np
import matplotlib
import pylab as pl
x = np.arange(0,1,0.1)
pl.plot(x,x)
pl.show()

On the first execution sometimes you need to execute the cell two times to see the graph.

If running the code you obtain to plot and get a message like:

>`/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3agg.py:16: UserWarning: The Gtk3Agg backend is known to not work on Python 3.x with pycairo. Try installing cairocffi. "The Gtk3Agg backend is known to not work on Python 3.x with pycairo. "`

Then you have problem that can be solved changing the matplotlib **backend** to pyqt5, for instance. Install it:

>`sudo apt-get install python3-pyqt5`

Then you need to instruct **matplotlib** to change the backend.  
You need where the configuration file is. The following code just gives this information:

In [ ]:
import matplotlib
print(matplotlib.matplotlib_fname())

Then edit the file and change the line that starts with **backend :** so that it contains:

>`backend : qt5agg`

As you can see, manual instalations can be complex. That's why **Anaconda** is recommeded for using **SLab**.

## Test the python installation

When you run the Python interpreter, for instance by executing python.exe in windows, you should get, in the case of the Anaconda installation, something like:

>`Python 3.5.1 |Anaconda custom (32-bit)| (default, Mar  4 2016, 15:28:01)  
[MSC v.1900 32 bit (Intel)] on win32  
Type "help", "copyright", "credits" or "license" for more information.`
>`>>>`

Refer to your preferred Python documentation if you cannot get to this point.

The "\>\>\>" is the prompt that ask you for commands to send.
Note that it says Python 3.5.1 But it could be another version.

Now it is time to verify that you have the needed packages. Write those lines (not the "\>\>\>" characters as they are written by the interpreter):

>`>>> import serial`  
>`>>> import jupyter`

If the interpreter keeps silent it means that both the **pyserial** and the **jupyter** modules are installed, so far, so good. Now we will check two more packages using an special import syntax:

>`>>> import numpy as np`  
>`>>> import pylab as pl`

If the interpreter keeps also silent it means that both **numpy** and **pylab**, that is part of **matplotlib**, are also installed, so far, so good. Now write down:

>`>>> x = np.arange(0,1,0.1)`  
>`>>> pl.plot(x,x)`

Python will respond with something like:

>`[<matplotlib.lines.Line2D object at 0x04EE6270>]`

Now write:

>`>>> pl.show()`  

You shall get an image like the one below:

![first plot](images\Board_01\first_plot.png)

Dismiss the image and you are done.
If you have reached to this point, you should be able to use the SLab System.

As this is an interactive **Jupyter notebook**, you can also run code from this window, so you can do the previous operations in a code cell instead of using a console.

**Run** the following cell to test all the imports and the operation of **NumPy** and **Matplotlib**. You should be able to obtain a figure with a line drawn.

If the cell responds with something like:

>`[<matplotlib.lines.Line2D object at 0x04EE6270>]`

Run the cell again.

In [ ]:
# Check the imports

import importlib

def check(package):
    try:
        importlib.import_module(package)
    except:
        print('Package',package,'is not installed (ERROR)')
    else:
        print('Package',package,'is installed (OK)')
        
check('serial')
check('numpy')
check('matplotlib')
check('jupyter')

# Check plot draing

import numpy as np
import pylab as pl
x = np.arange(0,1,0.1)
pl.plot(x,x)
pl.show()

## First Steps on SLab

If all four imports read '(OK)' and you are able to obtain a plot of a line, then the software requirements for **SLab** are fullfuled.

Now it is time to test the **SLab** code itself.

The SLab files are provided in a file **“Slab vX.Y(date).zip”** where X and Y are the major and minor versions of the release and **date** is the release date. Unzip this folder. We will refer to it as the SLab **root folder**. 

The **root folder** contains several subfolders.

* **Docs** is the location for static documentation like **PDF** files


* **Console** is the location to run SLab in console mode


* **Jupyter** is the location for interactive Jupyter notebook documents


* **Code** is the location where the SLab Python code resides. 


* **Boards** is the location for information about SLab hardware boards


There is a Readme.txt file in the root SLab folder that describes all folder contents.

Currently **SLab** is distributed as a set of **Python** files inside the `SLab/Code` folder, not as an installed Python **package**. That's why, if you want to access to the SLab code from outside of the **Code** folder you need to instruct Python about the SLab code location. This can be done using the **PYTHONPATH** environment variable.

So, if `path_to_code` is the path to the `SLab/Code` folder you can instruct Python to search for Python files in this folder in windows using:

>`set PYTHONPATH=path_to_code`

Or in Linux:

>`export PYTHONPATH=path_to_code`

You can also use relative paths when needed.

Using **SLab** you can normally access Python in three ways:

* Using the python console

* Using Jupyter notebooks

* Using Python scripts

Each mode is associated to a different folder in the SLab distribution.

## Running SLab from the Python console

To run slab from the **python console** just go to the `SLab\Console` folder.

Inside this file there are three executables: 

* Python_Console
* SLab_Console
* SLab_EZ

The executables are available in two favors: **.bat** files for Windows and **.sh** files for Linux. In the case of Linux it is possible that you need to make them executable using:

>`chmod 744 *.sh`

Use **SLab_Console** to open a new SLab Python console. The file loads the **start_console.py** file on start-up that imports the **SLab** module and sets the proper paths for SLab **calibration** data.

Use **SLab_EZ** to open a new SLab Python console in **easy mode** using the **slab.ez** module. The file load on start-up the **start.ez** module. See the reference documentation in the **SLab/Docs** folder for more details aboyt the EZ module.

Use the **Python_Console** to open a new Python console without loading or importing anything.

The **.bat** and **.sh** files should work well as they are, but depending on your system it is possible that you need to modify their contents.

The **Connection Details** section at the end of this documents give more details on how you connect with the hardware board.

You can interact with the board using a console, but, in general, the **Jupyter notebooks** provide a better interface.

## Running SLab from a Jupyter notebook

To run slab from a **Jupyter notebook** just go to the `SLab\Console` folder.

There you will find a **Jupyter.bat** file for Windows and a **Jupyter.sh** file for Linux. As in the console case it is possible that you need to set the **.sh** file as executable in Linux:

>`chmod 744 Jupyter.sh`

By running the file that matchs your operating system you will open **Jupyter** in your browser. That will open a window on your browser with a list of folders like in the figure below:

![Jupyter Folders](images/Board_01/Jupyter_Folders.png)

The **Sandbox** and **User** are your own folders. The rest of folders contain the **SLab** interactive documents for learning electronics. You can create more folders if you need them either by using the $New\rightarrow Folder$ Jupyter menu option or using operating system commands.

The **Sandbox** folder just contains a **Sandbox.ipynb** notebook with the code needed to import all **SLab** modules and connecting to the board. You can to this document as many cells as you want.

The **User** folder is just an empty folder. This is the place to put any new notebook you create.

## Running SLab from a Script

This is the most complex and powerful way to run the SLab system.

Just create new python scripts in the **SLab/Code** folder to improve the functions that **SLab** provides.

Your can import your new modules from the console or from Jupyter notebooks.

If you want to go this way that means that you know enough about Python so no more explanations are needed.

## Connection details

This section gives some details about how the **SLab** connects with the board. You can skip it if you don't want or need the details.

You can test the **SLab** conectivity from a Python console.

You can access **SLab** from the **SLab_Console** executable on the **SLab/Console** folder or by running **Python** from the **Slab/Code** folder. This last method, however, has the inconvenient that doesn't instruct **SLab** to use the proper **calibration** folder. Calibration was more important in the first **SLab** releases. Currently you can work totally fine on an uncalibrated board if you don't mind some small measurement errors.

Either way open the console.

In the case of **Linux**, it is possible that you need to access Python as **super user** by using **sudo** in order to be able to connect with the board. To do that you can use a line like:

>`sudo python`

You should not need to do that if you have configured your user in the **dialup** group as instructed previously.

After calling the Python interpreter you should get to the Python prompt. Now write the following line and press enter. Remember that the "\>\>\>" is written by Python, not by you.

>`>>> import slab`

Python should respond something like that:

>`SLAB Module
Version 1.0 (22/2/2017)
Running interactively`

The above import is not needed if you access the console by executing **SLab_Console** as this script automatically imports the **SLab** module.

If it gives any error, check that you have really executed the Python module executable from the proper location. 

You can execute the following **code cell** to do the same from this notebook:

In [ ]:
import slab

Now, we want to connect with the board. 

The **slab.py** module interacts with the board firmware using a serial **COM** port at **38400 baud** speed. For the connection to work, the drivers of the board shall be installed so it is detected by the system.

In order to obtain a successful connection you usually need to know the name of the port. In windows it is usually **"COMn"** where n is a number and you can usually request its number on Devices and Printers on the windows menu or in the device administration control panel. 
In Linux, the name can be something like **"/dev/ttyACM0"**.
As you have seen on a previous figure in this document, the name is **COM52** in my windows machine. In other operating systems knowing the port name will need a different method. You can refer to the Python [pyserial](https://pythonhosted.org/pyserial/) documentation or the operating system information to obtain more details.

Once you have the port name of the hardware board, input the following command on the Python console, changing **'COM52'** with whatever name your serial port has:

>`>>> slab.connect('COM52')`

In Linux it could be:

>`>>> slab.connect('/dev/ttyACM0')`

Python should respond with something like:

>`Getting board data
Connected to Nucleo64-F303RE SLab v1.0
No ADC calibration data found
No DAC calibration data found`

If the port name is not a proper port it will respond something like the following two lines following with some more exception information:

>`** SLab exception
** Cannot open connection. Check port`

If the port name exists on the system but it is not the board with the proper firmware loaded it will respond something like:

>`** Slab exception
** Board not responding. Check port and Firmware`

Or like:

>`** Slab exception
** Bad magic from board. Check Firmware`

In the first case there is not response from the com port after one second and in the second case the response from the board is wrong. The Slab board firmware contains a 4 byte magic code that guarantees that you only get a proper connection if you select a port that links to a board with the proper Slab firmware.

As you can see, the Python SLab module responds with an exception every time something goes wrong. That automatically aborts any running program. This is typical way to handle errors in Python.

If you are not able not obtain the proper name of the COM port used by the board, or you don't bother to find it, you can request the Slab module to autodetect the port. In order to do that, just request the connection without providing any port name:

>`>>>slab.connect()`

That should usually work on Windows, probably on Linux and I don't know on a Mac. If the board can be detected you will see something like:

>`Board detected at port COM52
Getting board data
Connected to Nucleo64-F303RE SLab v1.0
No ADC calibration data found
No DAC calibration data found`

You can use the following code to connect with the board:

In [ ]:
# Connect with the board using autodetection
slab.connect()

You can take note of the COM port nave for successive connections but the Python SLab module will also remember this port for you. The last successfully COM port opened is stored in a **“Last_COM.dat”** file. Check that this file is generated in the **Code** folder.

Next time the board tries to auto detect the port, it will first try to use the last valid COM port like in the next example:

>`>>> slab.connect()`

>`Trying last valid port COM52
Board detected
Getting board data
Connected to Nucleo64-F303RE SLab v1.0
No ADC calibration data found
No DAC calibration data found`

Using auto detect is always slower (although it is quite fast) than providing the proper port name. But thanks to the **“Last_COM.dat”** file the full auto detect algorithm will only need to be carried out once. Moreover, the “Last_COM.dat” file is also stored for manually selected ports, so, except in rare cases when autodetect fails for the first time, you can always use the connect command without arguments.

The auto detect algorithm works by requesting a magic code from all connected devices. As this is an active search, it can affect the normal operation of the devices in the system. If you detect any problem, refrain from using auto detect and manually select the port at least until the "Last_COM.dat" file is generated.

If a board cannot be detected automatically you will get an exception:

>`** SLab exception
** COM Autodetect Fail`

It usually means that there is a problem with the drivers, the STLINK firmware or the board SLab firmware. Off course it can also mean that the board is not connecte to the computer. You can check using a terminal emulator on the board port at 38400 baud that the boards reports its name when you reset it, but if this works, the SLab connection will usually work too.

It can also mean that you are already connected to the board with another Python interpreter. Remeber that you can only have one active connection between the board and a Python interpreter.

If the connection is established, you can disconnect from the board afterwards with the command:

>`>>> slab.disconnect()`

Python will respond with something like:

>`Disconnected from the Board`

You can test this using the code cell below:

In [ ]:
# Disconnect from the board
slab.disconnect()

As you have seen, and usual in python, we have used the slab module name for all the commands. All commands will have the structure:

>`var = slab.command_name(Arguments)`

Where **command_name** is the name of the command and **Arguments** is an optional argument list for the command. Some commands return values. In those cases you can assign the result to a variable **var** or, in some cases, a **list** of variables. For commands that do not return any value, or for the cases that the command returns data you don't need, you can omit the **"var ="** part of the call. As this is standard in Python refer to the Python literature for more information about this language.

It is not recommended to remove the slab reference by using:

>`>>> from slab import *`

It is bad Python practice and can break the operation of the program in unexpected ways. If you want to reduce the number of characters you are writing you'd better use an alias name for the module:

>`>>> import slab as sl`  
>`>>> sl.connect()`  
>`>>> sl.disconnect()`

Before leaving Python, both in interactive mode or using a script, it is recommended to disconnect from the board issuing:

>`>>> slab.disconnect()`

But probably Python will do that for you if you forget it.

## Getting Help

At any time you can ask SLab for help using the help command although it is probably more useful from the console than from a Jupyter notebook. The first **SLab** release used an specific help system. Nowadays, slab used the **standard** Python help system. You ask for information about a command just asking for it:

>`>>> help(slab.dcPrint)`

Like in the interactive box below:

In [ ]:
help(slab.connect)

Off course, as this is the standard Python help system, you can also ask about Python commands or about other module commands:

In [ ]:
help(print)

In [ ]:
import numpy as np
help(np.arange)

If you want, you can also call for help without arguments, that will open an **interactive help system**. Just hit return without specifiying any topic to close it.
Using the Python **interactive help** within **Jupyter** is not the best option, but it works.

In [ ]:
help()

At this point you have a nearly working **SLab system**. Next chapter will deal with building a **SLab system** by connecting a **hardware** board to **test circuit** on a **breadboard**.

## References

**STM32 Nucleo Page**  
Main page on the ST website about the STM32 Nucleo demonstration boards.  
http://www.st.com/en/evaluation-tools/stm32-mcu-nucleo.html

**MBED Developer Page**  
MBED developer page. This page gives access to the MBED compiler.  
https://developer.mbed.org/

**Python page:**  
Main page of the Python Software Foundation.  
https://www.python.org/

**Anaconda page:**  
Page where the Anaconda Python distribution can be downloaded.
https://www.continuum.io/downloads

**SLab Python References**  
Those are the [reference documents](http://localhost:8888/tree/Reference) for the SLab Python modules. They describe the commands that can be carried out after importing each module. 


## Document license

Copyright  ©  Vicente Jiménez (2018-2019)
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">